In [1]:
# Aadil Nazir
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import logging

# Logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


torch.manual_seed(42)

# Hyperparameters
batch_size = 32 # bcz i am working on CPU
learning_rate = 0.001 # You can incraesed Decrease LR acc... to Model Accuarcy
num_epochs = 5   # I started with less Epochs

# Data preparation with augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalization for CIFAR-10
])

# Download CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Device setup (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {device}")

# Model, Loss, and Optimizer setup
model = models.resnet18(weights=None)  
model.fc = nn.Linear(model.fc.in_features, 10)  
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # Reduce lr every 5 epochs


def clip_gradients(model, clip_value):
    for param in model.parameters():
        if param.grad is not None:
            param.grad.data.clamp_(-clip_value, clip_value)

# Training function
def train(model, train_loader, criterion, optimizer, scheduler, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 1):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Check for NaN loss
            if torch.isnan(loss):
                logging.error(f"NaN loss detected at epoch {epoch + 1}, batch {i}")
                return

            loss.backward()
            clip_gradients(model, clip_value=1.0)  
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 0:  # Log every 100 batches
                avg_loss = running_loss / 100
                logging.info(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{i}], Loss: {avg_loss:.4f}")
                running_loss = 0.0

        scheduler.step()  # Update learning rate
        logging.info(f"Evaluating after epoch {epoch + 1}...")
        evaluate(model, test_loader)

# Evaluate the model
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    logging.info(f"Test Accuracy: {accuracy:.2f}%")

# Start training
logging.info("Starting training...")
train(model, train_loader, criterion, optimizer, scheduler, num_epochs)

# Final evaluation
logging.info("Final evaluation on test data...")
evaluate(model, test_loader)


Files already downloaded and verified
Files already downloaded and verified


2024-12-14 13:07:58,210 - INFO - Using device: cpu
2024-12-14 13:07:58,647 - INFO - Starting training...
2024-12-14 13:09:12,084 - INFO - Epoch [1/5], Batch [100], Loss: 2.1142
2024-12-14 13:10:15,437 - INFO - Epoch [1/5], Batch [200], Loss: 1.8624
2024-12-14 13:11:20,916 - INFO - Epoch [1/5], Batch [300], Loss: 1.7984
2024-12-14 13:12:22,599 - INFO - Epoch [1/5], Batch [400], Loss: 1.7352
2024-12-14 13:13:24,197 - INFO - Epoch [1/5], Batch [500], Loss: 1.7251
2024-12-14 13:14:31,789 - INFO - Epoch [1/5], Batch [600], Loss: 1.6517
2024-12-14 13:15:26,400 - INFO - Epoch [1/5], Batch [700], Loss: 1.5811
2024-12-14 13:16:21,842 - INFO - Epoch [1/5], Batch [800], Loss: 1.6062
2024-12-14 13:17:18,447 - INFO - Epoch [1/5], Batch [900], Loss: 1.5384
2024-12-14 13:18:26,230 - INFO - Epoch [1/5], Batch [1000], Loss: 1.5518
2024-12-14 13:19:21,549 - INFO - Epoch [1/5], Batch [1100], Loss: 1.5201
2024-12-14 13:20:14,685 - INFO - Epoch [1/5], Batch [1200], Loss: 1.4574
2024-12-14 13:21:10,157 - IN

KeyboardInterrupt: 